In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy import signal
from tensorflow.keras import layers

# --- Data Loading and Preprocessing (Use your actual data loading) ---
# ... (Your data loading and preprocessing code here) ...
# For example, using dummy data like before.
def load_and_preprocess_ecg(file_path):
    df = pd.read_csv('C:/Users/abdulssekyanzi/EDA Dataset.csv/100.csv')
    ecg_signal = df['MLII'].values  # or 'V5'
    return ecg_signal

def preprocess_ecg_signal(ecg_signal):
    # Noise reduction (Butterworth filter)
    b, a = signal.butter(3, [0.5, 40], btype='bandpass', fs=360) #360 is the sample rate of the MIT-BIH dataset
    ecg_filtered = signal.lfilter(b, a, ecg_signal)

    # Standardization
    scaler = StandardScaler()
    ecg_scaled = scaler.fit_transform(ecg_filtered.reshape(-1, 1)).flatten()
    return ecg_scaled

def segment_heartbeats(ecg_signal, rpeaks):
    heartbeats = []
    for i in range(len(rpeaks) - 1):
        start = rpeaks[i] - 100  # Adjust as needed
        end = rpeaks[i + 1] + 100  # Adjust as needed
        if start >= 0 and end < len(ecg_signal):
            segment = ecg_signal[start:end]
            segment_resampled = signal.resample(segment, 250) #Resample to a fixed length
            heartbeats.append(segment_resampled)
    return np.array(heartbeats)

ecg_signal = np.random.randn(10000)
ecg_processed = preprocess_ecg_signal(ecg_signal)
rpeaks, _ = signal.find_peaks(ecg_processed, height=0.5)
heartbeats = segment_heartbeats(ecg_processed, rpeaks)
labels = np.random.randint(0, 5, size=len(heartbeats))
num_classes = len(np.unique(labels))
labels_one_hot = tf.keras.utils.to_categorical(labels, num_classes=num_classes)
X_train, X_temp, y_train, y_temp = train_test_split(heartbeats, labels_one_hot, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# --- Teacher Model (Improved) ---
teacher_model = tf.keras.Sequential([
    layers.Conv1D(128, 5, activation='relu', input_shape=(250, 1)),
    layers.MaxPooling1D(2),
    layers.Conv1D(256, 5, activation='relu'),
    layers.MaxPooling1D(2),
    layers.LSTM(256, return_sequences=True),
    layers.Dropout(0.5), # Added Dropout
    layers.LSTM(128),
    layers.Dropout(0.5), # Added Dropout
    layers.Dense(num_classes, activation='softmax')
])

teacher_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
teacher_model.fit(X_train[..., np.newaxis], y_train, epochs=30, validation_data=(X_val[..., np.newaxis], y_val)) #Increased epochs

# --- Student Model (Improved) ---
student_model = tf.keras.Sequential([
    layers.Conv1D(64, 3, activation='relu', input_shape=(250, 1)),
    layers.MaxPooling1D(2),
    layers.LSTM(128),
    layers.Dense(num_classes, activation='softmax')
])

# --- Knowledge Distillation ---
temperature = 5.0
alpha = 0.6 #Adjusted Alpha

def distillation_loss(teacher_logits, student_logits, temperature):
    teacher_probs = tf.nn.softmax(teacher_logits / temperature)
    student_probs = tf.nn.softmax(student_logits / temperature)
    return tf.keras.losses.KLDivergence()(teacher_probs, student_probs)

def combined_loss(y_true, y_pred, teacher_logits, student_logits, temperature, alpha):
    ce_loss = tf.keras.losses.CategoricalCrossentropy()(y_true, y_pred)
    dist_loss = distillation_loss(teacher_logits, student_logits, temperature)
    return alpha * dist_loss + (1 - alpha) * ce_loss

# Distillation Training Loop
optimizer = tf.keras.optimizers.Adam()

for epoch in range(30): #Increased Epochs
    with tf.GradientTape() as tape:
        teacher_logits = teacher_model(X_train[..., np.newaxis])
        student_logits = student_model(X_train[..., np.newaxis])
        loss = combined_loss(y_train, student_logits, teacher_logits, student_logits, temperature, alpha)

    gradients = tape.gradient(loss, student_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, student_model.trainable_variables))

    if epoch % 5 == 0:
        print(f"Epoch {epoch}, Loss: {loss.numpy()}")

# Compile the student model before evaluation
student_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# --- Evaluation ---
teacher_loss, teacher_accuracy = teacher_model.evaluate(X_test[..., np.newaxis], y_test)
student_loss, student_accuracy = student_model.evaluate(X_test[..., np.newaxis], y_test)

print(f"Teacher Accuracy: {teacher_accuracy}, Student Accuracy: {student_accuracy}")

C:\Users\abdulssekyanzi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 17s 608ms/step - accuracy: 0.2076 - loss: 1.6238 - val_accuracy: 0.1714 - val_loss: 1.6264
Epoch 2/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 498ms/step - accuracy: 0.2365 - loss: 1.6141 - val_accuracy: 0.1714 - val_loss: 1.6134
Epoch 3/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 430ms/step - accuracy: 0.1870 - loss: 1.6068 - val_accuracy: 0.1333 - val_loss: 1.6156
Epoch 4/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 433ms/step - accuracy: 0.2263 - loss: 1.6066 - val_accuracy: 0.1143 - val_loss: 1.6258
Epoch 5/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 11s 459ms/step - accuracy: 0.2056 - loss: 1.6058 - val_accuracy: 0.1810 - val_loss: 1.6217
Epoch 6/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 403ms/step - accuracy: 0.1955 - loss: 1.6040 - val_accuracy: 0.1524 - val_loss: 1.6184
Epoch 7/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 452ms/step - accuracy: 0.2101 - loss: 1.6043 - val_accuracy: 0.1524 - val_loss: 1.6372
Epoch 8/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 13s 631ms/step - accuracy: 0.2178 - loss: 1.6059 - val_accurac